In [85]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import math
import optuna
import joblib
import pickle
import chardet

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import load_linnerud
from sklearn.metrics import roc_auc_score,r2_score,mean_absolute_error,mean_squared_error,accuracy_score,classification_report,confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report,confusion_matrix
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso


In [86]:
data = pd.read_csv("est_delta_prediction_after_outlier_removal.csv")
data.head()

,title,platform,studio,genre,PreCovid,Covid,PostCovid,pitunes,pamazon,pvudu,...,first_seen_est_date,est_delta,SmallBudget,MediumBudget,LargeBudget,UnknownBudget,release_year,release_month,release_day,release_weekday
0,Red 11,VUDU,Other,"Horror,Sci-Fi,Suspense",1,0,0,0,0,1,...,2021-12-07,340,1,0,0,0,2021,1,1,4
1,Japanese Borscht,iTunes,Other,Action & Adventure,0,1,0,1,0,0,...,2020-06-16,322,1,0,0,0,2019,7,30,1
2,"""Homely"" What Men Want",Amazon Prime Video,Other,"Comedy, Drama",1,0,0,0,1,0,...,2019-08-17,317,0,0,0,1,2018,10,4,3
3,The Plagiarists,iTunes,Other,Comedy,0,1,0,1,0,0,...,2020-05-05,312,0,0,0,1,2019,6,28,4
4,Five More Minutes: Moments Like These,Google Play,Other,NaN,0,0,1,0,0,0,...,2023-10-24,311,0,0,0,1,2022,12,17,5


In [87]:
data.columns

Index(['title', 'platform', 'studio', 'genre', 'PreCovid', 'Covid',
       'PostCovid', 'pitunes', 'pamazon', 'pvudu', 'pappletv', 'pgoogle',
       'swb', 'ssony', 'suni', 'spara', 'sdis', 'sfox', 'shbo', 'sother',
       'release_date', 'first_seen_est_date', 'est_delta', 'SmallBudget',
       'MediumBudget', 'LargeBudget', 'UnknownBudget', 'release_year',
       'release_month', 'release_day', 'release_weekday'],
      dtype='object')

In [88]:
data_1 = data.drop(['title','platform','studio',
                   'genre','first_seen_est_date','sother'],axis = 1)
data_1.head()

,PreCovid,Covid,PostCovid,pitunes,pamazon,pvudu,pappletv,pgoogle,swb,ssony,...,release_date,est_delta,SmallBudget,MediumBudget,LargeBudget,UnknownBudget,release_year,release_month,release_day,release_weekday
0,1,0,0,0,0,1,0,0,0,0,...,2021-01-01,340,1,0,0,0,2021,1,1,4
1,0,1,0,1,0,0,0,0,0,0,...,2019-07-30,322,1,0,0,0,2019,7,30,1
2,1,0,0,0,1,0,0,0,0,0,...,2018-10-04,317,0,0,0,1,2018,10,4,3
3,0,1,0,1,0,0,0,0,0,0,...,2019-06-28,312,0,0,0,1,2019,6,28,4
4,0,0,1,0,0,0,0,1,0,0,...,2022-12-17,311,0,0,0,1,2022,12,17,5


In [89]:
data_1.shape

(43513, 25)

In [90]:
data_1['release_date'] =  pd.to_datetime(data_1['release_date'], errors = 'coerce')
data_1['release_year'] = data_1['release_date'].dt.year
data_1['release_month'] = data_1['release_date'].dt.month
data_1['release_day'] = data_1['release_date'].dt.day

In [91]:
data_1 = data_1.drop('release_date', axis = 1)

In [92]:
data_1['release_year'] = data_1['release_year'].astype(int)
data_1['release_month'] = data_1['release_month'].astype(int)
data_1['release_day'] = data_1['release_day'].astype(int)

In [93]:
def find_bounds(data_1):
    Q1 = data_1['est_delta'].quantile(0.25)
    Q3 = data_1['est_delta'].quantile(0.75)
    
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    new_data = data_1[(data_1['est_delta'] >= 0) & (data_1['est_delta'] <= 45)]
    return new_data

In [94]:
data_1 = find_bounds(data_1)

In [95]:
# data_1 = data_1.drop('pitunes', axis = 1)
# LR: 32, DT: 54 , RF: 70 ,Ridge: 32 ,KNN: 52

In [96]:
# data_1 = data_1.drop('pvudu', axis = 1)
# LR: 32, DT: 51 , RF: 70 ,Ridge: 32 ,KNN: 52

In [97]:
# data_1 = data_1.drop('pappletv', axis = 1)
# LR: 32, DT: 55 , RF: 70 ,Ridge: 32 ,KNN: 52

In [98]:
# data_1 = data_1.drop('pgoogle', axis = 1)
# LR: 32, DT: 56 , RF: 69 ,Ridge: 32 ,KNN: 52

In [99]:
# data_1 = data_1.drop('pamazon', axis = 1)
# LR: 32, DT: 47 , RF: 66 ,Ridge: 32 ,KNN: 51

In [100]:
# data_1 = data_1.drop(['pitunes','pvudu','pappletv','pgoogle'], axis = 1)
# LR: 32, DT: 58 , RF: 67 ,Ridge: 32 ,KNN: 53

In [101]:
#data_1 = data_1.drop(['pamazon','pitunes','pappletv','pgoogle'], axis = 1)
# LR: 20, DT: 43 , RF: 52 ,Ridge: 20 ,KNN: 39

In [102]:
# data_1 = data_1.drop(['pamazon','pvudu','pitunes','pgoogle'], axis = 1)
# LR: 19, DT: 40 , RF: 49 ,Ridge: 19 ,KNN: 37

In [103]:
# data_1 = data_1.drop(['pamazon','pvudu','pappletv','pitunes'], axis = 1)
# LR: 19, DT: 43 , RF: 53 ,Ridge: 19 ,KNN: 38

In [104]:
#data_1 = data_1.drop(['pamazon','pvudu','pappletv','pgoogle'], axis = 1)
# LR: 19, DT: 36 , RF: 49 ,Ridge: 19 ,KNN: 37

In [105]:
data_1.shape

(36813, 24)

In [106]:
X = data_1.drop(['est_delta'], axis=1)
y = data_1['est_delta']

In [107]:
def Linear_regression(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Mean Absolute Error: {mae}")
    print(f"Root Mean Squared Error: {mse**0.5}")
    print(f"R-squared: {r2*100}")

In [108]:
def Decision_tree(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Mean Absolute Error: {mae}")
    print(f"Root Mean Squared Error: {mse**0.5}")
    print(f"R-squared: {r2*100}")

In [109]:
def Random_forest(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators = 200, random_state = 42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Mean Absolute Error: {mae}")
    print(f"Root Mean Squared Error: {mse**0.5}")
    print(f"R-squared: {r2*100}")

In [110]:
def Ridge_(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = Ridge()
    model = Ridge(alpha = 10.0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Mean Absolute Error: {mae}")
    print(f"Root Mean Squared Error: {mse**0.5}")
    print(f"R-squared: {r2*100}")

In [111]:
def Knn_(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = KNeighborsRegressor(n_neighbors = 5, metric = 'minkowski')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Mean Absolute Error: {mae}")
    print(f"Root Mean Squared Error: {mse**0.5}")
    print(f"R-squared: {r2*100}")

In [112]:
print('Linear Regression:\n')
Linear_regression(X,y)
print('\nDecision Tree Regresor:\n')
Decision_tree(X,y)
print('\nRandom Forest Regressor:\n')
Random_forest(X,y)
print('\nRidge Regression:\n')
Ridge_(X,y)
print('\nKNN:\n')
Knn_(X,y)

Linear Regression:

Mean Absolute Error: 5.8467197752381
Root Mean Squared Error: 9.087213479160702
R-squared: 16.115618409276912

Decision Tree Regresor:

Mean Absolute Error: 4.623837455737541
Root Mean Squared Error: 9.097855646483007
R-squared: 15.919026932600023

Random Forest Regressor:

Mean Absolute Error: 4.647262234746852
Root Mean Squared Error: 8.248917188320814
R-squared: 30.87843968396957

Ridge Regression:

Mean Absolute Error: 5.850168459469212
Root Mean Squared Error: 9.085721127205442
R-squared: 16.143168051834557

KNN:

Mean Absolute Error: 5.024989813934537
Root Mean Squared Error: 8.935849359497672
R-squared: 18.88683954084258


In [287]:
def objective(trial):
    # Define the hyperparameter search space
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 200),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10)
    }
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and train the model with the suggested hyperparameters
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
#     print(f"Mean Absolute Error: {mae}")
#     print(f"Root Mean Squared Error: {mse**0.5}")
#     print(f"R-squared: {r2*100}")

    return mae  # Optuna minimizes the objective function, so return the metric to be minimized



In [288]:
# Create a study object and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2024-03-13 00:29:41,170] A new study created in memory with name: no-name-372a9596-dfa3-4246-9236-dae0cdc8070e
[I 2024-03-13 00:29:56,213] Trial 0 finished with value: 11.049537393939403 and parameters: {'n_estimators': 171, 'max_depth': 15, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 0 with value: 11.049537393939403.
[I 2024-03-13 00:30:01,620] Trial 1 finished with value: 11.801379984697414 and parameters: {'n_estimators': 76, 'max_depth': 12, 'min_samples_split': 2, 'min_samples_leaf': 2}. Best is trial 0 with value: 11.049537393939403.
[I 2024-03-13 00:30:12,930] Trial 2 finished with value: 11.28240550945092 and parameters: {'n_estimators': 141, 'max_depth': 14, 'min_samples_split': 5, 'min_samples_leaf': 2}. Best is trial 0 with value: 11.049537393939403.
[I 2024-03-13 00:30:23,242] Trial 3 finished with value: 11.896717068884561 and parameters: {'n_estimators': 139, 'max_depth': 14, 'min_samples_split': 7, 'min_samples_leaf': 7}. Best is trial 0 with value: 

[I 2024-03-13 00:34:00,590] Trial 36 finished with value: 11.543353012118617 and parameters: {'n_estimators': 59, 'max_depth': 14, 'min_samples_split': 3, 'min_samples_leaf': 4}. Best is trial 33 with value: 10.532316533995344.
[I 2024-03-13 00:34:07,272] Trial 37 finished with value: 11.259289788858565 and parameters: {'n_estimators': 77, 'max_depth': 15, 'min_samples_split': 3, 'min_samples_leaf': 3}. Best is trial 33 with value: 10.532316533995344.
[I 2024-03-13 00:34:16,952] Trial 38 finished with value: 10.545246934191395 and parameters: {'n_estimators': 89, 'max_depth': 19, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 33 with value: 10.532316533995344.
[I 2024-03-13 00:34:21,910] Trial 39 finished with value: 12.611045206903896 and parameters: {'n_estimators': 88, 'max_depth': 10, 'min_samples_split': 4, 'min_samples_leaf': 6}. Best is trial 33 with value: 10.532316533995344.
[I 2024-03-13 00:34:32,319] Trial 40 finished with value: 11.721080156666188 and paramet

[I 2024-03-13 00:38:28,935] Trial 72 finished with value: 11.771669486596227 and parameters: {'n_estimators': 55, 'max_depth': 20, 'min_samples_split': 5, 'min_samples_leaf': 9}. Best is trial 54 with value: 10.121458366730373.
[I 2024-03-13 00:38:36,581] Trial 73 finished with value: 10.42349866259425 and parameters: {'n_estimators': 71, 'max_depth': 18, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 54 with value: 10.121458366730373.
[I 2024-03-13 00:38:44,385] Trial 74 finished with value: 10.1647650216205 and parameters: {'n_estimators': 67, 'max_depth': 20, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 54 with value: 10.121458366730373.
[I 2024-03-13 00:38:51,434] Trial 75 finished with value: 10.263702923292575 and parameters: {'n_estimators': 62, 'max_depth': 19, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 54 with value: 10.121458366730373.
[I 2024-03-13 00:38:59,720] Trial 76 finished with value: 10.150070113864615 and parameters

In [289]:
# Get the best parameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'n_estimators': 63, 'max_depth': 20, 'min_samples_split': 4, 'min_samples_leaf': 1}


In [290]:
X = data_1.drop(['est_delta'], axis=1)
y = data_1['est_delta']

In [291]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [292]:
# Get the best hyperparameters from the Optuna study
best_params = {'n_estimators': 63, 'max_depth': 20, 'min_samples_split': 4, 'min_samples_leaf': 1}

# Create a RandomForestRegressor model with the best hyperparameters
best_model = RandomForestRegressor(**best_params)

# Train the model on your training data
best_model.fit(X_train, y_train)

opt_rsq = best_model.score(X_test, y_test)

# Predict the target values using the best model
y_pred = best_model.predict(X_test)

opt_mse = mean_squared_error(y_test, y_pred)
opt_mae = mean_absolute_error(y_test, y_pred)
opt_rmse = np.sqrt(opt_mse)

In [293]:
print('\nHyperparameter Tuned Optuna:\n')
print('\nR2_Score:', round(opt_rsq * 100, 2), '%')
print('Mean Squared Error:', round(opt_mse, 2))
print('Mean_Absolute_Error:', round(opt_mae, 2))
print('Root_Mean_Squared_Error:', round(opt_rmse, 2))


Hyperparameter Tuned Optuna:


R2_Score: 69.34 %
Mean Squared Error: 375.33
Mean_Absolute_Error: 10.14
Root_Mean_Squared_Error: 19.37


In [ ]:
plt.figure(figsize = (18, 8))

x = ['Linear Regression', 'Decision Tree', 'Random Forest', 'Ridge', 'K-nearest Neighbors','Optuna']
y = [lr_mae, dtr_mae, rf_mae, ridge_mae, knn_mae, lasso_mae, opt_mae] 

# Sort both x and y values based on y-values
sorted_xy = sorted(zip(y, x), reverse = True)
y_sorted = [i[0] for i in sorted_xy]
x_sorted = [i[1] for i in sorted_xy]

x1 = np.array(x_sorted)
y1 = np.array(y_sorted)

a = sns.barplot(x = x1, y = y1, palette = 'plasma')

for i in a.containers:
    a.bar_label(i, )

plt.xlabel('\nAlgorithms')
plt.ylabel('r2_score (in %)') 
plt.show()